### Validating Areal Interpolation

##### Outlier Neighborhood Analysis

Look at `spatial_operations_validation.ipynb` for a broader overview of areal interpolation results. Here, we are going to look at two of the outlier neighborhoods to see if we can get a better sense of what is going on.

Specifically, we are going to look at Edgewater and South Shore. Our areal interpolation methods differed significantly from our validation data for both; the tract-to-neighborhood crosswalk method differed significantly from validation data for South Shore but not at all for Edgewater.

In [6]:
import os
import sys
sys.path[0] = os.path.join(os.path.abspath(''),'..')  # make sure we can import from our package

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
from data_pipeline.fetch_census_data import acs5_aggregate 
from data_pipeline import spatial_operations as so
from data_pipeline import interactive_mapping as im
import numpy as np
import pandas as pd
import geopandas as gpd
from IPython.display import display

In [ ]:
outputs = pd.read_csv('validation_data').set_index('community_area')

In [ ]:
outputs.loc['EDGEWATER']

In [ ]:
outputs.loc['SOUTH SHORE']

First, we will compare maps of these neighborhoods with tract maps.

In [ ]:
neighborhoods = so.geographize(pd.DataFrame(columns=['community_area']),'community_area').set_index('community_area')


In [ ]:
edgewater = neighborhoods.loc['EDGEWATER']
edgewater_area = pd.DataFrame(columns=edgewater.index)
edgewater_area.loc[0] = edgewater.values
edgewater_area['community_area'] = 'EDGEWATER'
south_shore = neighborhoods.loc['SOUTH SHORE']
south_shore_area = pd.DataFrame(columns=south_shore.index)
south_shore_area.loc[0] = south_shore.values
south_shore_area['community_area'] = 'SOUTH SHORE'


In [ ]:
so.simple_map(gpd.GeoDataFrame(edgewater_area),'area',target_geography = 'community_area')

In [ ]:
edgewater_map = im.InteractiveMap(gpd.GeoDataFrame(edgewater_area),target_geography='community_area')
edgewater_map.map

In [ ]:
# importing Census tracts mapped to community area number 
# renaming columns

tracts = pd.read_csv("../data/tracts_comm_areas.csv",
                        index_col=0,
                        parse_dates=[0]).rename(columns={
                        "COMMAREA": "comm_num",          
                        "TRACTCE10": "tract"})

# importing community area numbers and names
# renaming columns

comm_area = pd.read_csv("../data/comm_areas.csv",
                        index_col=0,
                        parse_dates=[0]).rename(columns={
                        "AREA_NUMBE": "comm_num"})

# merging both dataframes above to map tract wirh community area name

tract_area= comm_area.merge(tracts, on='comm_num')

# selecting columns we need and renaming them

tract_area=tract_area[['comm_num', 'tract']]

# 801 rows x 3 columns
# final dataframe

tract_area.head()

In [ ]:
neighborhoods = neighborhoods.rename(columns={'area_numbe':'comm_num'}).reset_index()
neighborhoods['comm_num'] = neighborhoods['comm_num'].astype(np.int64)
crosswalk = tract_area.merge(neighborhoods,on='comm_num')
crosswalk.head()

In [ ]:
crosswalk.groupby('community_area').get_group('EDGEWATER').tract

This does exactly match the visually-found census tracts for the area. None of the census tracts seem to overlap any other neighborhood areas.